In [6]:
import pandas as pd
import numpy as np


from openprompt.data_utils.utils import InputExample
from openprompt.data_utils.data_processor import DataProcessor
from sklearn.model_selection import train_test_split
## self-modified
from pipeline_base_modified import PromptDataLoader, PromptForClassification
from template_generation_wzy import LMBFFTemplateGenerationTemplate, T5TemplateGenerator
from openprompt.plms import load_plm
from tqdm import tqdm
import torch.nn as nn
import os
from openprompt.prompts import ManualVerbalizer
from openprompt.prompts import ManualTemplate
from openprompt.trainer import ClassificationRunner
import copy
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup


from openprompt import PromptForClassification
from openprompt.plms import load_plm
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
from openprompt.prompts import ManualTemplate



In [7]:
def sum_list(a,b):
    res = []
    if len(a) >= len(b):
        for i in range(len(b)):
            res.append(a[i]+b[i])# 
        for i in range(len(a) - len(b)):
            res.append(a[len(b)+i])
        return res
    elif len(a) < len(b):
        return sum_list(b, a)


os.environ['CUDA_VISIBLE_DEVICES'] = '0'
use_cuda = True
plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-large")


In [14]:
Personalities = ['A','C','E','O','N']

for personality in Personalities:
    with open('label_words/' + personality + '_words.txt', 'r') as f:
        pos = f.readline().split(',')
        neg = f.readline().split(',')

    with open('label_words/' + personality + '_weights.txt', 'r') as f:
        pos_weights = eval(f.readline())
        neg_weights = eval(f.readline())
    
    
    candidate_templates = []
    datasets = ['Friends', 'MyPersonality', 'Essay', 'Pan']
    for dataset in datasets:
        with open('templates/Adapted_t5_large_'+dataset+'_'+personality+'_templates_top_10.txt', 'r') as f:
            candidate_templates += [i.strip() for i in f.readlines()]
    
    classes = [0,1]        
    myverbalizer = ManualVerbalizer(
        classes = classes,
        label_words = {
            0 : neg, 
            1 : pos
        },
        tokenizer=tokenizer)

    from openprompt import PromptDataLoader
    batch_size = 1

    overall_logits = []

    for i in range(len(candidate_templates)):
        sample = InputExample(guid=i, text_a="", label=0)
        mytemplate = ManualTemplate(
            text = candidate_templates[i],
            tokenizer = tokenizer,
        )
        prompt_model = PromptForClassification(plm=plm, template=mytemplate, verbalizer=myverbalizer, freeze_plm=True)
        if use_cuda:
            prompt_model = prompt_model.cuda()
        prompt_model.eval()
        validation_dataloader = PromptDataLoader(dataset=[sample], template=mytemplate, tokenizer=tokenizer,
            tokenizer_wrapper_class=WrapperClass, max_seq_length=128,
            batch_size=batch_size, shuffle=False, teacher_forcing=False, predict_eos_token=False,
            truncate_method="head")

        for step, inputs in enumerate(validation_dataloader):
            if use_cuda:
                inputs = inputs.cuda()
            logits = prompt_model.forward_without_verbalize(inputs)
            label_words_logits = prompt_model.verbalizer.project(logits)
            label_words_probs = prompt_model.verbalizer.normalize(label_words_logits)
            label_words_logits = label_words_probs # torch.log(label_words_probs+1e-15)
            overall_logits.append(label_words_logits.detach())

    overall_logits = torch.cat(overall_logits)
    overall_logits = torch.mean(overall_logits, 0)

    neg_logits = [0.1*float(i) for i in list(overall_logits[0]/overall_logits[0].sum().cpu())]
    pos_logits = [0.1*float(i) for i in list(overall_logits[1]/overall_logits[1].sum().cpu())]

    pos_weights_ = [i/sum(pos_weights) for i in pos_weights]
    neg_weights_ = [i/sum(neg_weights) for i in neg_weights]


    pos_logit_weight = sum_list(pos_weights_, pos_logits)
    neg_logit_weight = sum_list(neg_weights_, neg_logits)



    pos_dict = {}
    for word, weight in zip(pos, pos_logit_weight[:len(pos)]):
        pos_dict[word] = weight

    neg_dict = {}
    for word, weight in zip(neg, neg_logit_weight[:len(neg)]):
        neg_dict[word] = weight  

    n = 100
    pos_ = sorted(pos_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)[:n]
    neg_ = sorted(neg_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)[:n]
    
    print('Top', n, 'positive label words are:', [i[0] for i in pos_])
    print('Top', n, 'negative label words are:', [i[0] for i in neg_])

    with open('label_words/posterior_'+personality+'_label_words.txt', 'w') as f:
        f.write(','.join([i[0] for i in pos_]))
        f.write(','.join([i[0] for i in neg_]))
    with open('label_words/posterior_'+personality+'_label_weights.txt', 'w') as f:
        f.write(str([i[1] for i in pos_]))
        f.write('\n')
        f.write(str([i[1] for i in neg_]))





tokenizing: 1it [00:00, 2534.32it/s]
tokenizing: 1it [00:00, 2790.62it/s]
tokenizing: 1it [00:00, 2395.38it/s]
tokenizing: 1it [00:00, 2334.06it/s]
tokenizing: 1it [00:00, 2403.61it/s]
tokenizing: 1it [00:00, 2439.97it/s]
tokenizing: 1it [00:00, 2366.99it/s]
tokenizing: 1it [00:00, 2465.79it/s]
tokenizing: 1it [00:00, 1827.58it/s]
tokenizing: 1it [00:00, 2437.13it/s]
tokenizing: 1it [00:00, 2418.86it/s]
tokenizing: 1it [00:00, 2332.76it/s]
tokenizing: 1it [00:00, 2371.00it/s]
tokenizing: 1it [00:00, 2376.38it/s]
tokenizing: 1it [00:00, 1524.09it/s]
tokenizing: 1it [00:00, 2326.29it/s]
tokenizing: 1it [00:00, 2231.01it/s]
tokenizing: 1it [00:00, 2420.26it/s]
tokenizing: 1it [00:00, 1762.31it/s]
tokenizing: 1it [00:00, 2385.84it/s]
tokenizing: 1it [00:00, 2420.26it/s]
tokenizing: 1it [00:00, 2432.89it/s]
tokenizing: 1it [00:00, 2240.55it/s]
tokenizing: 1it [00:00, 2267.19it/s]
tokenizing: 1it [00:00, 1510.37it/s]
tokenizing: 1it [00:00, 2290.72it/s]
tokenizing: 1it [00:00, 1646.76it/s]
t

Top 100 positive label words are: ['good-humoured', 'onefold', 'likeable', 'nice', 'hot', 'sadistic', 'kind', 'freehearted', 'sympathetic', 'likable', 'congenial', 'easy', 'derivative', 'fine', 'warmly', 'warm', 'easy-going', 'simpleton', 'simple-minded', 'simple', 'lukewarm', 'great', 'polite', 'courteous', 'civil', 'gracious', 'polished', 'gallant', 'affable', 'helpful', 'prepared', 'heat', 'genteel', 'baking', 'warmth', 'aflutter', 'sincere', 'warm hearted', 'solemn', 'tisane', 'sentimental', 'liqueur', 'large-minded', 'heartfelt', 'elixir', 'cordial', 'cheer', 'invigorating', 'enjoyable', 'serious-minded', 'patient of', 'nesh', 'sensitive', 'medium', 'sensible', 'aware', 'tolerant', 'caring', 'beneficial', 'charitable', 'agreeable', 'resistant', 'welfare', 'humanitarian', 'eleemosynary', 'merciful', 'conformable', 'pleasant', 'happy', 'smooth', 'hard-nosed', 'pleasing', 'free-and-easy', 'patient', 'peaceful', 'patriotic', 'affected role', 'loyal', 'trustful', 'womanly', 'sweet', 'f

tokenizing: 1it [00:00, 2737.80it/s]
tokenizing: 1it [00:00, 2455.68it/s]
tokenizing: 1it [00:00, 2914.74it/s]
tokenizing: 1it [00:00, 2439.97it/s]
tokenizing: 1it [00:00, 2400.86it/s]
tokenizing: 1it [00:00, 2413.29it/s]
tokenizing: 1it [00:00, 2366.99it/s]
tokenizing: 1it [00:00, 2857.16it/s]
tokenizing: 1it [00:00, 1633.93it/s]
tokenizing: 1it [00:00, 2379.07it/s]
tokenizing: 1it [00:00, 2404.99it/s]
tokenizing: 1it [00:00, 2473.06it/s]
tokenizing: 1it [00:00, 2384.48it/s]
tokenizing: 1it [00:00, 2406.37it/s]
tokenizing: 1it [00:00, 2444.23it/s]
tokenizing: 1it [00:00, 2272.10it/s]
tokenizing: 1it [00:00, 1541.46it/s]
tokenizing: 1it [00:00, 2273.34it/s]
tokenizing: 1it [00:00, 1691.93it/s]
tokenizing: 1it [00:00, 2377.72it/s]
tokenizing: 1it [00:00, 2308.37it/s]
tokenizing: 1it [00:00, 2391.28it/s]
tokenizing: 1it [00:00, 2299.51it/s]
tokenizing: 1it [00:00, 2275.80it/s]
tokenizing: 1it [00:00, 1572.08it/s]
tokenizing: 1it [00:00, 2322.43it/s]
tokenizing: 1it [00:00, 2359.00it/s]
t

Top 100 positive label words are: ['truehearted', 'sadistic', 'organized', 'organised', 'accurate', 'on time', 'sad', 'precise', 'exact', 'sweetheart', 'hard work', 'hard', 'hard-and-fast', 'hardworking', 'hard working', 'right winger', 'fastidious', 'hard-nosed', 'authentic', 'secure', 'reliable', 'canny', 'exacting', 'logical', 'brief', 'straightaway', 'acquisitive', 'consistent', 'measured', 'concerned', 'legitimate', 'careful', 'succinct', 'concise', 'prompt', 'coherent', 'attentive', 'command prompt', 'anxious', 'lucid', 'mindful', 'terse', 'motivate', 'solicitous', 'work', 'strong', 'positive', 'regular', 'reproducible', 'steady', 'meticulous', 'painstaking', 'steadily', 'brace', 'punctilious', 'unfluctuating', 'unfaltering', 'decided', 'decisive', 'conclusive', 'reckful', 'simpleton', 'simple-minded', 'simple', 'difficult', 'tauten', 'free', 'timely', 'solid', 'productive', 'responsible for', 'responsible', 'company', 'firm', 'enterprise', 'punctual', 'corporation', 'unwaveringl

tokenizing: 1it [00:00, 2294.48it/s]
tokenizing: 1it [00:00, 2427.26it/s]
tokenizing: 1it [00:00, 2371.00it/s]
tokenizing: 1it [00:00, 2389.92it/s]
tokenizing: 1it [00:00, 2360.33it/s]
tokenizing: 1it [00:00, 2351.07it/s]
tokenizing: 1it [00:00, 2357.68it/s]
tokenizing: 1it [00:00, 2347.12it/s]
tokenizing: 1it [00:00, 2303.30it/s]
tokenizing: 1it [00:00, 1741.10it/s]
tokenizing: 1it [00:00, 2423.05it/s]
tokenizing: 1it [00:00, 2417.47it/s]
tokenizing: 1it [00:00, 2341.88it/s]
tokenizing: 1it [00:00, 2353.71it/s]
tokenizing: 1it [00:00, 1373.83it/s]
tokenizing: 1it [00:00, 2377.72it/s]
tokenizing: 1it [00:00, 2239.35it/s]
tokenizing: 1it [00:00, 2290.72it/s]
tokenizing: 1it [00:00, 1611.95it/s]
tokenizing: 1it [00:00, 2437.13it/s]
tokenizing: 1it [00:00, 2369.66it/s]
tokenizing: 1it [00:00, 2348.43it/s]
tokenizing: 1it [00:00, 2331.46it/s]
tokenizing: 1it [00:00, 2259.86it/s]
tokenizing: 1it [00:00, 2304.56it/s]
tokenizing: 1it [00:00, 2216.86it/s]
tokenizing: 1it [00:00, 2431.48it/s]
t

Top 100 positive label words are: ['happy', 'happy-go-lucky', 'good-humoured', 'truehearted', 'sadistic', 'long winded', 'bigmouthed', 'loud', 'brazen', 'extraverted', 'noisy', 'strong-growing', 'funny', 'noise', 'extravertive', 'vocal', 'sounded', 'shout', 'extroverted', 'extrospective', 'extrorse', 'talkative', 'drivellous', 'chatty', 'verbose', 'garrulous', 'loquacious', 'prolix', 'aggressive', 'dauntless', 'bellicose', 'daredevil', 'bad-mannered', 'sociable', 'exciting', 'social', 'going strong', 'up-and-coming', 'nice', 'likeable', 'boldface', 'bold', 'non finite verb', 'verbal', 'bluff', 'audacious', 'assertive', 'lectic', 'sicker', 'enthusiastic', 'self assured', 'confident', 'sure-footed', 'hot', 'attractive', 'mettlesome', 'enlivened', 'spirited', 'bouncing', 'communicative', 'hard work', 'hard', 'appealing', 'simple', 'energetic', 'charismatic', 'magnetic', 'magnetised', 'free', 'gay', 'cute', 'easy-going', 'shiny', 'boldness', 'rambunctious', 'daring', 'straight-from-the-sho

tokenizing: 1it [00:00, 2882.68it/s]
tokenizing: 1it [00:00, 2418.86it/s]
tokenizing: 1it [00:00, 2380.42it/s]
tokenizing: 1it [00:00, 2403.61it/s]
tokenizing: 1it [00:00, 2395.38it/s]
tokenizing: 1it [00:00, 2400.86it/s]
tokenizing: 1it [00:00, 2355.03it/s]
tokenizing: 1it [00:00, 2348.43it/s]
tokenizing: 1it [00:00, 2339.27it/s]
tokenizing: 1it [00:00, 2399.49it/s]
tokenizing: 1it [00:00, 2430.07it/s]
tokenizing: 1it [00:00, 2447.09it/s]
tokenizing: 1it [00:00, 2387.20it/s]
tokenizing: 1it [00:00, 2416.07it/s]
tokenizing: 1it [00:00, 1576.21it/s]
tokenizing: 1it [00:00, 1464.49it/s]
tokenizing: 1it [00:00, 2269.65it/s]
tokenizing: 1it [00:00, 1529.09it/s]
tokenizing: 1it [00:00, 2232.20it/s]
tokenizing: 1it [00:00, 2407.75it/s]
tokenizing: 1it [00:00, 2389.92it/s]
tokenizing: 1it [00:00, 2437.13it/s]
tokenizing: 1it [00:00, 2290.72it/s]
tokenizing: 1it [00:00, 2332.76it/s]
tokenizing: 1it [00:00, 2331.46it/s]
tokenizing: 1it [00:00, 2323.71it/s]
tokenizing: 1it [00:00, 1559.80it/s]
t

Top 100 positive label words are: ['good-humoured', 'complicated', 'great', 'sad', 'sadistic', 'truehearted', 'well-informed', 'free', 'intelligent', 'level-headed', 'silly', 'hardworking', 'hard working', 'smart', 'funny', 'atypical', 'complex', 'likeable', 'intellectual', 'cerebral', 'voguish', 'freehearted', 'ache', 'building complex', 'bad-mannered', 'philosophical', 'philosophic', 'brilliant', 'deep', 'subtle', 'selfish', 'bright', 'promising', 'low', 'heavy', 'innovative', 'forward-looking', 'adroit', 'artful', 'unconventional', 'ready', 'undimmed', 'burnished', 'lustrous', 'fulgent', 'propitious', 'glimmering', 'brightly', 'luminiferous', 'innovatory', 'innovational', 'gifted', 'ingenious', 'deeply', 'able', 'fertile', 'knowledgeable', 'bass', 'inscrutable', 'agog', 'trench', 'recondite', 'sagacious', 'inquisitive', 'versed', 'insightful', 'nonconforming', 'analytical', 'analytic', 'perspicacious', 'introspective', 'important', 'meditative', 'contemplative', 'perceptive', 'artic

tokenizing: 1it [00:00, 2449.94it/s]
tokenizing: 1it [00:00, 2939.25it/s]
tokenizing: 1it [00:00, 2467.24it/s]
tokenizing: 1it [00:00, 2451.38it/s]
tokenizing: 1it [00:00, 2366.99it/s]
tokenizing: 1it [00:00, 2304.56it/s]
tokenizing: 1it [00:00, 2325.00it/s]
tokenizing: 1it [00:00, 1527.98it/s]
tokenizing: 1it [00:00, 2477.44it/s]
tokenizing: 1it [00:00, 2418.86it/s]
tokenizing: 1it [00:00, 2444.23it/s]
tokenizing: 1it [00:00, 2409.13it/s]
tokenizing: 1it [00:00, 2416.07it/s]
tokenizing: 1it [00:00, 2296.99it/s]
tokenizing: 1it [00:00, 2285.72it/s]
tokenizing: 1it [00:00, 2348.43it/s]
tokenizing: 1it [00:00, 2316.02it/s]
tokenizing: 1it [00:00, 2289.47it/s]
tokenizing: 1it [00:00, 2383.13it/s]
tokenizing: 1it [00:00, 2373.69it/s]
tokenizing: 1it [00:00, 2418.86it/s]
tokenizing: 1it [00:00, 2322.43it/s]
tokenizing: 1it [00:00, 2328.88it/s]
tokenizing: 1it [00:00, 2308.37it/s]
tokenizing: 1it [00:00, 2291.97it/s]
tokenizing: 1it [00:00, 2259.86it/s]
tokenizing: 1it [00:00, 2273.34it/s]
t

Top 100 positive label words are: ['aflutter', 'bad-tempered', 'sad', 'overjealous', 'important', 'touchy', 'moody', 'Helen Newington Wills', 'glowering', 'Dwight Lyman Moody', 'easy', 'thin-skinned', 'sadistic', 'ticklish', 'temperamental', 'emotional', 'worked up', 'nettlesome', 'effusive', 'irritable', 'stressed', 'jealous', 'green with envy', 'justificatory', 'ineffectual', 'envious', 'covetous', 'overcautious', 'possessive', 'genitive', 'raring', 'fidgety', 'fretful', 'querulous', 'impatient', 'selfish', 'agent', 'macrophobic', 'self-pitying', 'nervous', 'female', 'emasculated', 'high-strung', 'ratty', 'crabby', 'queasy', 'skittish', 'defensive', 'truculent', 'cantankerous', 'grumpy', 'surly', 'grouchy', 'insecure', 'frightening', 'unsafe', 'cranky', 'dangerous', 'faultfinding', 'anxious', 'frightened', 'unquiet', 'hardline', 'disturbed', 'fearful', 'uneasy', 'quarrelsome', 'captious', 'frightful', 'angstful', 'womanly', 'feminine', 'aware', 'afraid', 'nesh', 'about', 'stupid', 's